In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pandas as pd
from pyspark.sql.window import Window
import time
from pyspark.sql import SparkSession

/tmp/ipykernel_52920/1510302545.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
spark = SparkSession.builder \
       .master("local[*]") \
       .appName("test") \
       .config("spark.driver.memory", "80g")\
       .config("spark.executor.memory", "50g")\
       .config("spark.dirver.maxResultSize", "50g")\
       .config('spark.local.dir', '/dados10t/datalake/raw/checkpoint_dir')\
       .getOrCreate()
sc = spark.sparkContext
sc.setCheckpointDir('/dados10t/datalake/raw/checkpoint_dir')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/20 17:00:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/20 17:00:42 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/10/20 17:00:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Mapeamento

In [4]:
df_sintomas =\
spark.read.csv('/dados10t/datalake/standard/omop/resultado_mapeamento/srag-condition/sintomas_mapeados.csv', header=True)

In [5]:
df_comorbidades =\
spark.read.csv('/dados10t/datalake/standard/omop/resultado_mapeamento/srag-condition/comorbidades_mapeadas.csv', header=True)

In [6]:
conditions = df_comorbidades.union(df_sintomas)

In [7]:
conditions = conditions.withColumn('condition', F.col('source_code_description'))

In [7]:
#
conditions.columns

['source_code',
 'source_concept_id',
 'source_vocabulary_id',
 'source_code_description',
 'target_concept_id',
 'target_vocabulary_id',
 'valid_start_date',
 'valid_end_date',
 'invalid_reason',
 'condition']

In [ ]:
#
conditions\
.select('source_code_description', 'target_concept_id')\
.show(100)

# Srag

In [8]:
# srag = spark.read.parquet("/dados10t/datalake/standard/vigivac/srag/dbsrag_tb_influd_long_09072022_v8.parquet/")
srag = spark.read.parquet('/dados10t/datalake/standard/omop/data_with_ids/dbsrag_tb_influd_long_09072022_v8-id_omop-v3.parquet/')

In [9]:
id_vigivac = spark.read.parquet("/dados10t/datalake/standard/vigivac/base_analise/id_paciente_to_id_vigivac/id_vigivac_id_paciente_09072022.parquet/")

In [10]:
srag = srag.join(id_vigivac, ['id_paciente'], 'inner')

In [228]:
#
srag.printSchema()

root
 |-- id_paciente: string (nullable = true)
 |-- st_notific: string (nullable = true)
 |-- dt_notific: string (nullable = true)
 |-- sem_not: string (nullable = true)
 |-- dt_sin_pri: string (nullable = true)
 |-- sem_pri: string (nullable = true)
 |-- sg_uf_not: string (nullable = true)
 |-- id_regiona: string (nullable = true)
 |-- co_regiona: decimal(10,0) (nullable = true)
 |-- id_municip: string (nullable = true)
 |-- co_mun_not: string (nullable = true)
 |-- id_unidade: string (nullable = true)
 |-- co_uni_not: string (nullable = true)
 |-- cs_sexo: string (nullable = true)
 |-- dt_nasc: string (nullable = true)
 |-- nu_idade_n: decimal(10,0) (nullable = true)
 |-- tp_idade: string (nullable = true)
 |-- cod_idade: string (nullable = true)
 |-- cs_gestant: decimal(10,0) (nullable = true)
 |-- cs_raca: string (nullable = true)
 |-- cs_etinia: string (nullable = true)
 |-- cs_escol_n: decimal(10,0) (nullable = true)
 |-- nu_cep: string (nullable = true)
 |-- id_pais: string (nu

### comorbidades

In [11]:
aberto_comorbidade = spark.read.parquet('/dados10t/datalake/standard/omop/extracao_termos/srag/id_row/campo_aberto_comorbidades.parquet')

In [12]:
aberto_comorbidade_columns = aberto_comorbidade.columns
aberto_comorbidade_columns.remove('id_vigvac')
aberto_comorbidade_columns.remove('id_row')
aberto_comorbidade_columns.remove('dt_notific')
aberto_comorbidade_columns.remove('morb_desc')
aberto_comorbidade_columns.remove('outro_des')

In [13]:
aberto_comorbidade = aberto_comorbidade.withColumn('conditions', 
              F.array(
                  [F.when(F.col(e)==1, e) for e in aberto_comorbidade_columns]
                     )
             ).withColumn('conditions', F.array_compact('conditions'))

In [14]:
aberto_comorbidade = aberto_comorbidade.select( 'id_row' , F.explode_outer('conditions').alias('condition'))

In [15]:
aberto_comorbidade  = aberto_comorbidade.withColumn('condition', F.regexp_replace(F.col('condition'), '_o$', ''))

### sintomas

In [16]:
aberto_sintomas = spark.read.parquet('/dados10t/datalake/standard/omop/extracao_termos/srag/id_row/campo_aberto_sintomas.parquet')

In [17]:
aberto_sintomas_columns = aberto_sintomas.columns
aberto_sintomas_columns.remove('dt_notific')
aberto_sintomas_columns.remove('id_vigvac')
aberto_sintomas_columns.remove('id_row')
aberto_sintomas_columns.remove('outro_des')
aberto_sintomas_columns.remove('morb_desc')

In [18]:
aberto_sintomas = aberto_sintomas.withColumn('conditions', 
              F.array(
                  [F.when(F.col(e)==1, e) for e in aberto_sintomas_columns]
                     )
             ).withColumn('conditions', F.array_compact('conditions'))

In [19]:
aberto_sintomas = aberto_sintomas.select( 'id_row' , F.explode_outer('conditions').alias('condition'))

In [20]:
aberto_sintomas = aberto_sintomas.withColumn('condition', F.regexp_replace(F.col('condition'), '_o$', ''))

### concat data

In [21]:
conditions_df = aberto_sintomas.union(aberto_comorbidade)

In [22]:
conditions_mapped = conditions_df.join(conditions, ['condition'], 'left').filter(F.col('condition').isNotNull())

In [23]:
#
conditions_mapped.filter(F.col('source_code_description').isNull()).groupBy(F.col('condition')).count().show()

23/10/20 17:01:21 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+------+
|           condition| count|
+--------------------+------+
|                dpoc| 26796|
|              rinite|  1198|
|doenca renal cronica|    28|
|            enfisema|  2375|
|           pneumonia|  1322|
|                 has|285814|
|    hipertireoidismo|  1715|
+--------------------+------+



In [24]:
conditions_mapped = conditions_mapped.select('id_row', 'target_concept_id').filter(F.col('target_concept_id').isNotNull())

### join data

In [25]:
srag_mapeado = srag.join(conditions_mapped, ['id_row'], 'inner')

##### selecao var interesse

In [26]:
srag_mapeado = srag_mapeado.select(['id_vigvac', 'visit_occurrence_id', 'dt_interna', 'dt_notific', 'target_concept_id'])

In [27]:
srag_mapeado = srag_mapeado.withColumn("condition_start_date", 
                                       F.coalesce(
                                           F.to_date(F.col('dt_interna'), 'dd/MM/yyyy'), 
                                           F.to_date(F.col('dt_notific'), 'dd/MM/yyyy') 
                                       )
                                      )

In [28]:
srag_mapeado = srag_mapeado.filter(F.col('condition_start_date').isNotNull())

In [26]:
# we need target_concept_id
# dpoc is missing, what to do?
srag_mapeado.filter(F.col('condition_start_date').isNotNull()).count()/srag_mapeado.count()

23/10/11 20:01:19 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


1.0

### build structure

In [ ]:
#
srag_mapeado.printSchema()

In [29]:
srag_mapeado = srag_mapeado.withColumnRenamed('target_concept_id', 'condition_concept_id')

In [30]:
srag_mapeado = srag_mapeado.select('id_vigvac', 'visit_occurrence_id', 'condition_start_date', 'condition_concept_id')

In [197]:
srag_mapeado.columns

['id_vigvac',
 'visit_occurrence_id',
 'condition_start_date',
 'condition_concept_id']

# SG

In [31]:
# sg = spark.read.parquet("/dados10t/datalake/standard/vigivac/SG/sg_long_09072022_v9.parquet/")
sg = spark.read.parquet('/dados10t/datalake/standard/omop/data_with_ids/sg_long_09072022_v9-id_omop.parquet/')

In [199]:
#
sg.printSchema()

root
 |-- dt_datanotificacao: date (nullable = true)
 |-- dt_datainiciosintomas: date (nullable = true)
 |-- dt_datateste: date (nullable = true)
 |-- condicoes: string (nullable = true)
 |-- sintomas: string (nullable = true)
 |-- classificacaofinal: string (nullable = true)
 |-- dt_dataencerramento: date (nullable = true)
 |-- evolucaocaso: string (nullable = true)
 |-- tipoteste: string (nullable = true)
 |-- resultadoteste: string (nullable = true)
 |-- codigotipoteste: string (nullable = true)
 |-- codigoresultadoteste: string (nullable = true)
 |-- estadonotificacao: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: double (nullable = true)
 |-- codigo_ibge_resid: string (nullable = true)
 |-- profissionalsaude: string (nullable = true)
 |-- cbo: string (nullable = true)
 |-- evolucaocaso_2: string (nullable = true)
 |-- classificacaofinal_2: string (nullable = true)
 |-- sexo_2: string (nullable = true)
 |-- racac

In [ ]:
#
conditions.select('condition').show(100, truncate=False)

#### selecao var interesse

In [32]:
sg = sg.select('id_vigvac', 'visit_occurrence_id', 'dt_datanotificacao', 'dt_datainiciosintomas', 'sintomas', 'condicoes')

In [33]:
from collections import defaultdict

### cleaning sintomas

In [34]:
sg = sg\
.withColumn('sintomas_split', F.split('sintomas', ','))\
.withColumn('sintomas_split', F.explode(F.column('sintomas_split')))\
.withColumn('sintomas_split', F.lower(F.trim('sintomas_split')))\
.filter(F.col('sintomas_split') != '')\
.filter(F.col('sintomas_split').isNotNull())

In [35]:
#
sg.select('sintomas_split').groupBy('sintomas_split').count().orderBy(F.desc('count')).show(truncate=False)

+------------------------------+--------+
|sintomas_split                |count   |
+------------------------------+--------+
|tosse                         |36424385|
|outros                        |27450503|
|dor de garganta               |27055974|
|dor de cabeça                 |26461410|
|febre                         |24347056|
|coriza                        |21163193|
|assintomático                 |16737995|
|dispneia                      |8437573 |
|distúrbios gustativos         |5239557 |
|distúrbios olfativos          |4920400 |
|outros: paciente assintomático|154634  |
|dispineia                     |4498    |
|dificuldade de respirar       |989     |
|tossedificuldade de respirar  |635     |
|cefaleia                      |227     |
|febredor de garganta          |215     |
|dispnéia                      |168     |
|fraqueza                      |103     |
|adinamia                      |103     |
|diarreia                      |86      |
+------------------------------+--

In [36]:
@F.udf(returnType=StringType())
def map_sintomas(sintoma):
    map_dict = {
        'tosse': 'tosse',
        'outros': None,
        'dor de garganta': 'garganta',
        'dor de cabeça': 'cefaleia',
        'febre': 'febre',
        'coriza': 'coriza',
        'assintomático': None,
        'dispneia': 'dispneia',
        'distúrbios gustativos': 'perd_pala',
        'distúrbios olfativos': 'perd_olft',
        'outros: paciente assintomático': None,
        'dispineia': 'dispneia',
        'dificuldade de respirar': 'dispneia',
    }
    map_dict = defaultdict(lambda: None, map_dict)
    return map_dict[sintoma]

In [37]:
sg = sg.withColumn('sintomas_cleaned', map_sintomas(F.col('sintomas_split')))

In [ ]:
#
sg.select('sintomas_cleaned').groupBy('sintomas_cleaned').count().orderBy(F.desc('count'))\
.withColumn('percent', F.col('count')/F.sum('count').over(Window.partitionBy())*100)\
.show(truncate=False)

### tratamento condicoes

In [38]:
sg = sg\
.withColumn('condicoes_split', F.split('condicoes', ','))\
.withColumn('condicoes_long', F.explode(F.column('condicoes_split')))\
.withColumn('condicoes_long', F.lower(F.trim('condicoes_long')))\
.filter(F.col('condicoes_long') != '')\
.filter(F.col('condicoes_long').isNotNull())

In [39]:
#
sg.groupBy('condicoes_long').count().orderBy(F.desc('count')).show(100, truncate=False)

+----------------------------------------------------------------------------------+-------+
|condicoes_long                                                                    |count  |
+----------------------------------------------------------------------------------+-------+
|doenças cardíacas crônicas                                                        |8543609|
|diabetes                                                                          |5115115|
|doenças respiratórias crônicas descompensadas                                     |4614036|
|outros                                                                            |2188102|
|obesidade                                                                         |1572126|
|gestante                                                                          |1133421|
|imunossupressão                                                                   |967933 |
|4 ou 5)                                                              

In [40]:
@F.udf(returnType=StringType())
def map_condicoes(condicao):
    map_dict = {
        'doenças cardíacas crônicas': 'cardiopata',
        'diabetes': 'diabetes',
        'doenças respiratórias crônicas descompensadas': 'pneumopati', #pneumopatia
        'outros': None,
        'obesidade': 'obesidade',
        'gestante': None,
        'imunossupressão': 'imunodepre',
        'doenças renais crônicas em estágio avançado (graus 3': 'renal',
        'portador de doenças cromossômicas ou estado de fragilidade imunológica': 'imunodepre',
        'doenças cardíacas crônicas descompensadas': 'cardiopata',
        'doenças renais crônicas': 'renal',
        'doenças cardoacas crônicas': 'cardiopata',
        'doenças renais crônicas ou em estágio avançado (graus 3': 'renal',
        'portador de doenças cromossômicas ou estado de fragilidade': 'imunodepre',
        'doença cardiovascular': 'cardiopata'
    }
    map_dict = defaultdict(lambda: None, map_dict)
    return map_dict[condicao]

In [41]:
sg = sg.withColumn('condicoes_cleaned', map_condicoes(F.col('condicoes_long')))

In [ ]:
# 
sg.groupBy('condicoes_cleaned')\
.count().orderBy(F.desc('count'))\
.withColumn('percent', F.col('count')/F.sum('count').over(Window.partitionBy())*100)\
.show(100, truncate=False)

##### join condicoes + sintomas

In [42]:
sg = sg\
.withColumn('condition', F.array(F.col('sintomas_cleaned'), F.col('condicoes_cleaned')))\
.withColumn('condition', F.explode_outer('condition'))\
.filter(F.col('condition').isNotNull())

In [ ]:
#
sg.filter(F.col('condition').isNull()).count()

### mapeamento

In [43]:
sg_mapeado = sg.join(conditions, 'condition', 'left')

### build structure

In [ ]:
sg_mapeado.printSchema()

In [44]:
sg_mapeado = sg_mapeado\
.withColumn("condition_start_date", F.coalesce(F.col("dt_datainiciosintomas"), F.col("dt_datanotificacao") ))\
.withColumnRenamed('target_concept_id', 'condition_concept_id')

In [45]:
sg_mapeado = sg_mapeado.filter(F.col('condition_start_date').isNotNull())

In [46]:
sg_mapeado = sg_mapeado.select('id_vigvac', 'visit_occurrence_id', 'condition_start_date', 'condition_concept_id')

In [ ]:
sg_mapeado.printSchema()

# union SG+SRAG

In [47]:
sg_srag = sg_mapeado.union(srag_mapeado)

In [48]:
sg_srag = sg_srag.checkpoint()

In [217]:
sg_srag.distinct().count()

36895473

In [218]:
sg_srag.count()

67601332

In [49]:
sg_srag = sg_srag.distinct()

In [235]:
sg_srag.printSchema()

root
 |-- id_vigvac: long (nullable = true)
 |-- visit_occurrence_id: long (nullable = true)
 |-- condition_start_date: date (nullable = true)
 |-- condition_concept_id: string (nullable = true)
 |-- person_id: long (nullable = true)
 |-- condition_type_concept_id: integer (nullable = false)



## person_id 

In [50]:
person_id = spark.read.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_person.parquet/")\
.select('person_id', 'id_vigvac')

In [51]:
sg_srag = sg_srag.join(person_id, 'id_vigvac', 'inner')

## condition_type_concept_id 

In [52]:
sg_srag = sg_srag.withColumn('condition_type_concept_id', F.lit(32817))

## condition_occurrence_id  

In [49]:
# full_id = sg_srag.withColumn("new_column", F.lit("ABC"))
# w = Window().partitionBy('new_column').orderBy(F.lit('A'))
# full_id_with_row = full_id.withColumn("condition_occurrence_id",F.row_number().over(w)).drop("new_column").checkpoint()

In [53]:
full_id_with_row = sg_srag.rdd.zipWithIndex().toDF().select(F.col("_1.*"),F.col("_2").alias('condition_occurrence_id'))

In [229]:
full_id_with_row.describe().show()

+-------+--------------------+--------------------+--------------------+--------------------+-------------------------+-----------------------+
|summary|           id_vigvac| visit_occurrence_id|condition_concept_id|           person_id|condition_type_concept_id|condition_occurrence_id|
+-------+--------------------+--------------------+--------------------+--------------------+-------------------------+-----------------------+
|  count|            67601332|            67601332|            67601332|            67601332|                 67601332|               67601332|
|   mean|8.549695098373966E11|2.5263576198494256E8|   3739213.146207341| 9.420135634837797E7|                  32817.0|           3.38006655E7|
| stddev|4.959038378632853E11| 2.455860063588613E8|   9271859.504480254|5.4414195757757865E7|                      0.0|     1.95148237582262E7|
|    min|                   7|                   0|              196523|                   2|                    32817|                 

In [54]:
full_id_with_row = full_id_with_row\
.withColumn('visit_occurrence_id', F.col('visit_occurrence_id').cast(IntegerType()))\
.withColumn('condition_concept_id', F.col('condition_concept_id').cast(IntegerType()))\
.withColumn('person_id', F.col('person_id').cast(IntegerType()))\
.withColumn('condition_type_concept_id', F.col('condition_type_concept_id').cast(IntegerType()))\
.withColumn('condition_occurrence_id', F.col('condition_occurrence_id').cast(IntegerType()))

### writing to disk

##### data

In [55]:
full_id_with_row.drop('id_vigvac').write.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_condition_occurence_data.parquet")

23/10/20 18:04:01 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /dados10t/datalake/raw/checkpoint_dir/blockmgr-65875f42-1649-42d6-81d2-90c921737bd3. Falling back to Java IO way
java.io.IOException: Failed to delete: /dados10t/datalake/raw/checkpoint_dir/blockmgr-65875f42-1649-42d6-81d2-90c921737bd3
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:177)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:113)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:94)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1231)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1(DiskBlockManager.scala:368)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1$adapted(DiskBlockManager.scala:364)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptim